In [4]:
import spacy
import en_core_web_sm
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

## read dataset
df=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
df.columns=['statement_ID','label','statement','subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','context']

## remove stop words
stop=stopwords.words('english')
df['statement']=df['statement'].apply(lambda x: "".join([word for word in x.split() if word not in (stop)]))

nlp=en_core_web_sm.load()
df.columns.values

# nn_phr=[]
# for i in range(0,len(df)):
#     sent=nlp(df['statement'][i])
#     res=list(sent.noun_chunks)
#     nn_phr.append(res)

# final_res=[]
# for noun_phrase in nn_phr:
#     res=noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_))
#     final_res.append(res)


array(['statement_ID', 'label', 'statement', 'subject', 'speaker',
       'job_title', 'state_info', 'pantry_affiliation', 'barely_true_cnt',
       'false_cnt', 'half_true_cnt', 'mostly_true_cnt',
       'pants_on_fire_cnt', 'context'], dtype=object)

### Helper function to extract key target phrases in fake news statement


In [ ]:
def get_noun_Phrase(doc):
    assert len(doc)>=0
    nn_phrase, nn_chunks = [],[]
    for i in range(0, len(df)):
        sent = nlp(df['statement'][i])
        res = list(sent.noun_chunks)
        nn_chunks.append(res)
    for noun_phrase in nn_chunks:
        res = noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)
        nn_phrase.append(res)
    return nn_phrase

## test the function
dataset=df.loc[1:10,]
dataset.head()

get_noun_Phrase(dataset)

### Helper function to extract noun-adjective phrase in fake news statement

In [ ]:
def get_noun_adj(doc):
    nlp=en_core_web_sm.load()
    noun_adjs=[]
    for i in range(0, len(doc)):
        sent = nlp(df['statement'][i])
        for j, tok in enumerate(sent):
            if tok.pos_ not in ('NOUN','PRON'):
                continue
            for k in range(i+1, len(sent)):
                if sent[j].pos_=='ADJ':
                    noun_adjs.append(tok, sent[j])
                    break
    return noun_adjs

## test helper function
dataset=df.loc[1:10,]
get_noun_adj(dataset)


### Helper function to extract compound noun adjectives

In [ ]:
def get_compound_nn_adj(doc):
    compounds_nn_pairs = []
    for i in range(0, len(doc)):
        sent = nlp(doc['statement'][i])
        compounds=[token for token in sent if token.dep_=='compound']
        compounds=[nc for nc in compounds if nc.i==0 or sent[nc.i-1].dep_!='compound']
        if compounds:
            for token in compounds:
                pair_1, pair_2=(False, False)
                noun=sent[token.i:token.head.i+1]
                pair_1=noun
                if noun.root.dep_=='nsubj':
                    adj_list=[rt for rt in noun.root.head.rights if rt.pos_=='ADJ']
                    if adj_list:
                        pair_2=adj_list[0]
                if noun.root.dep_=='dobj':
                    verb_root=[vb for vb in noun.root.ancestors if vb.pos_=='VERB']
                    if verb_root:
                        pair_2=verb_root[0]
                if pair_1 and pair_2:
                    compounds_nn_pairs.append(pair_1, pair_2)
        else:
            return False
    return compounds_nn_pairs

## test the helper function
dataset=df.loc[1:10,]
get_compound_nn_adj(dataset)